In [1]:
%load_ext autoreload

In [2]:
import xlrd
import pandas as pd

from datetime import datetime as dt
from datetime import timedelta

from model import plan_var as V

from plotly.subplots import make_subplots
import plotly.graph_objects as go

from gantt_bar import *

import holidays

In [22]:
h = holidays.RUS()
dt(2020,7,12)


datetime.datetime(2020, 7, 12, 0, 0)

In [3]:
# List1
List1 = ['javascript', 'csharp', 'go', 'c', 'c++']
 
# List2
List2 = ['go', 'python', 'fuck']

check = any(item in List1 for item in List2)
check

True

In [4]:
%autoreload
plan_loader = PlanLoader('план.xlsx')
plan_loader.delete_rows_ready()

NameError: name 'PlanLoader' is not defined

In [5]:
plan_loader.df.head(20)

NameError: name 'plan_loader' is not defined

In [6]:
%autoreload
ref = PlanReformatter(plan_loader.df)
ref.r_df

NameError: name 'PlanReformatter' is not defined

In [7]:
%autoreload
l = ref.get_column_names(True)
l

NameError: name 'ref' is not defined

In [8]:
from gantt_bar import get_color

In [9]:
s = set()
s.add(5)
s.add(1)
s.add(1)
1 not in s

False

In [10]:
li = [l1 for l1 in range(11)]
li[-1:None]

[10]

In [11]:
%autoreload
dd = data = ref.get_df_for_gantt(2,1100)
dd

NameError: name 'ref' is not defined

In [12]:
%autoreload
fig = go.Figure()
b = GanttBar()

data = ref.get_df_for_gantt(17,10)
for i in range(data.shape[0]):
    fig.add_trace(b.get_bar(data['Start'][i], data['Finish'][i],
                    l.index(data['Task'][i]), group=int(data['zakaz'][i])))
    
fig.update_layout(
    yaxis = dict(
        tickmode = 'array',
        tickvals = [n for n in range(len(l))],
        ticktext = l
    )
)

fig.update_xaxes(
    dtick='D1',
    range=[dt.today() - timedelta(days=7),
          dt.today() + timedelta(days=30),]
    )
fig.show()

NameError: name 'go' is not defined

In [13]:
dt.today() - timedelta(days=10)

datetime.datetime(2020, 9, 11, 12, 4, 47, 820685)

In [14]:
data[data['Task'] == 'УсткаRM']

NameError: name 'data' is not defined